In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

import matplotlib.pyplot as plt
import numpy as np

# functions to show an image

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(10, 20, 5)
        self.fc1 = nn.Linear(20 * 5 * 5, 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 20 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

import torch.optim as optim
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)
if torch.cuda.device_count() > 1:
    print('\n===> Training on GPUS!')
    net = nn.DataParallel(net)


criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(15):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))


class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Files already downloaded and verified
Files already downloaded and verified
[1,  2000] loss: 2.125
[1,  4000] loss: 1.725
[1,  6000] loss: 1.549
[1,  8000] loss: 1.500
[1, 10000] loss: 1.434
[1, 12000] loss: 1.376
[2,  2000] loss: 1.312
[2,  4000] loss: 1.288
[2,  6000] loss: 1.233
[2,  8000] loss: 1.223
[2, 10000] loss: 1.205
[2, 12000] loss: 1.154
[3,  2000] loss: 1.075
[3,  4000] loss: 1.043
[3,  6000] loss: 1.040
[3,  8000] loss: 1.034
[3, 10000] loss: 1.030
[3, 12000] loss: 1.020
[4,  2000] loss: 0.873
[4,  4000] loss: 0.893
[4,  6000] loss: 0.896
[4,  8000] loss: 0.890
[4, 10000] loss: 0.906
[4, 12000] loss: 0.903
[5,  2000] loss: 0.748
[5,  4000] loss: 0.760
[5,  6000] loss: 0.767
[5,  8000] loss: 0.795
[5, 10000] loss: 0.789
[5, 12000] loss: 0.782
[6,  2000] loss: 0.596
[6,  4000] loss: 0.651
[6,  6000] loss: 0.659
[6,  8000] loss: 0.681
[6, 10000] loss: 0.682
[6, 12000] loss: 0.699
[7,  2000] loss: 0.494
[7,  4000] loss: 0.544
[7,  6000] loss: 0.563
[7,  8000] loss: 0.571
[7, 